In [238]:
import pickle
import pandas as pd
import numpy as np

In [239]:
with open('data/18.08-3.09.pkl', 'rb') as f:
    data = pickle.load(f)

In [240]:
data_dict = {}
idx = 0
for match in data:
    location = match['location']
    date = match['date']
    teams = []
    teams = match['teams']
    team_0 = teams[0]['team']
    country_0 = teams[0]['county']
    team_1 = teams[1]['team']
    country_1 = teams[1]['county']
    for game in match['maps']:
        picks_0 = game['picks'][0]
        picks_1 = game['picks'][1]
        bans_0 = game['bans'][0]
        bans_1 = game['bans'][1]
        winner = game['winner']
        data_dict[idx] = {
            'location': location,
            'team_0': team_0,
            'team_1': team_1,
            'country_0': country_0,
            'country_1': country_1,
            'picks_0': picks_0,
            'bans_0': bans_0,
            'picks_1': picks_1,
            'bans_1': bans_1,
            'winner': winner
        }
        idx += 1
        
df = pd.DataFrame.from_dict(data_dict, orient='index')

In [241]:
df = df[df.picks_0.apply(lambda x: '' not in x)].reset_index(drop=True)

In [242]:
len(df)

459

In [243]:
df_heroes = pd.concat([df_picks_0, df_picks_1, df_bans_0, df_bans_1], axis=1)
all_heroes = set(df_heroes.stack().tolist())
all_heroes.remove('')
all_heroes = sorted(all_heroes)

del(df_heroes)

In [244]:
class FewHotEncoder:
    def __init__(self, words):
        self.list_of_words = sorted(set(words))
        self.empty_array = np.zeros(len(self.list_of_words))
    def encode(self, words):
        encoded = self.empty_array.copy()
        for word in words:
            encoded[self.list_of_words.index(word)] = 1
        return encoded
            

In [245]:
enc = FewHotEncoder(all_heroes)

In [246]:
all_heroes = np.array(all_heroes)

In [247]:
encoded_picks_0 = df.picks_0.apply(lambda x: enc.encode(x).astype(int))
encoded_picks_1 = df.picks_1.apply(lambda x: enc.encode(x).astype(int))

encoded_picks = encoded_picks_0 - encoded_picks_1
encoded_picks = list(encoded_picks.apply(list))
encoded_picks = pd.DataFrame(encoded_picks)

In [248]:
len(encoded_picks)

459

In [249]:
df = df.drop(columns=['picks_0', 'picks_1', 'bans_0', 'bans_1'])

In [251]:
df = pd.concat([df, encoded_picks], axis=1)

In [252]:
df.location.unique()

array(['Southeast Asia', 'South America', 'Europe/CIS', 'China', 'Asia',
       '', 'Americas'], dtype=object)

In [254]:
df.head()

,location,team_0,team_1,country_0,country_1,winner,0,1,2,3,...,108,109,110,111,112,113,114,115,116,117
0,Southeast Asia,AG,R5,Indonesia,Indonesia,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Southeast Asia,AG,R5,Indonesia,Indonesia,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,South America,Inter,BINO,Brazil,Brazil,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,South America,Inter,BINO,Brazil,Brazil,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Europe/CIS,Unique,SPigz,CIS,Europe,1,0,0,0,0,...,0,-1,0,0,0,0,1,0,0,0


In [253]:
df.country_0.unique()

array(['Indonesia', 'Brazil', 'CIS', 'Peru', 'Philippines', 'Russia',
       'Belarus', 'China', 'Malaysia', 'Myanmar', 'Thailand',
       'Kazakhstan', 'Europe', 'North America', 'World', 'Sweden', 'USA',
       'South Korea', 'Mongolia'], dtype=object)